In [3]:
import sqlite3
import pandas as pd
import re

df = sqlite3.connect('/Users/MIGUEL/Desktop/CLAB1/Ironhack-Module-1-Project---Pipelines-/data/raw/Miguel318.db')

dfRI = pd.read_sql_query("SELECT * FROM rank_info", df)
dfRI

#Wrangling rank_info
    #eliminating columns that do not add value (Unnamed:0)

dfRI.drop(['Unnamed: 0'], axis='columns', inplace=True)

    #Wrangling - "position column"

print(dfRI.position.isnull().sum())   #Only 52 NaN values, not proceed delete column

dfRI['position'] = dfRI['position'].fillna(0)   #replance NaN values with 0.0
dfRI['position'] = dfRI['position'].astype('float').astype('int')  #replacement float values with int


#convert all characters to lowercase in column name

dfRI = dfRI.apply(lambda x: x.astype(str).str.lower())


#analyzing types of values to see the next cleaning stage
     #we can verify that we have duplicate elements so we must eliminate them but taking care that we eliminate the correct one and that it contributes less value to the dataframe. 

dfRI

52


,id,name,position
0,3239,anant asavabhokhin & family,0
1,9391,arash ferdowsi,0
2,9523,banthoon lamsam & family,0
3,3154,bill adderley & family,0
4,8503,boris zingarevich,0
...,...,...,...
2255,1711,zhao xiaoqiang,2204
2256,3538,zhou liangzhang,2205
2257,7797,zhu xingming,2206
2258,3123,zhuo jun,2207
